In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [18]:
#  This function finds the percentage of each value compared to the previous one
#  For the first value, since there is not a previous one, I find the difference between the first element and the second one
def AvgPrevious(nparray):
    originalarray = nparray
    print(originalarray)
    for i in range(len(nparray)):
        #print(originalarray[i])
        if i > 0:
            nparray[i] = (originalarray[i] - originalarray[i - 1])
            print(originalarray[i] - originalarray[i - 1])
        else:
            nparray[i] = (originalarray[i] - originalarray[i + 1])
    return nparray

In [19]:
rawdata = pd.read_csv('data/bitcoin.csv',usecols=[3,8])
data = rawdata.as_matrix()
#print(np.argwhere(data=='2016-10-08')[0][0])

res = getEvalData(data,'2015-10-15',predict_delta)

evaluation = np.array([res[0]])
#print(evaluation)
AvgPrevious(evaluation[0])
#model.predict(evaluation)
#print(rawdata[0].loc['2013-04-28'])

895
[244.94 247.05 245.31 249.51 251.99]
251.27000000000004
-253.01000000000005
257.21000000000004
-254.73000000000002


array([ -2.11, 249.16,  -3.85, 253.36,  -1.37])

In [7]:
# This function retrieves the previous 4 days given a single day for input from the .csv file.
def getEvalData(nparray,date,predict_delta):
    res = [[]]
    idx = np.argwhere(nparray==date)[0][0] - 5
    print(idx)
    # if the given date does not have 5 previous values...
    if idx < 4 or idx + 7 > nparray.shape[0]:
        return
    else:
        for i in range(5):
            res[0].append(nparray[i+idx][1])
        predict_day = nparray[i+idx+predict_delta][1]
        predict_day_percent = (predict_day - res[0][4])
        res.append(predict_day_percent)
        res.append(predict_day)
    return res
        

In [6]:
# Trying to convert the input data, which starts off as dataframeX, must take 5 days and then transpose it
# and then convert it into a numpy array

rawdata = pd.read_csv('data/bitcoin.csv',usecols=[8])
days = 5                                                      # Number of days we are looking at
predict_delta = 7                                             # Number of days we are predicting into the future 
number_of_items = rawdata.shape[0] - (predict_delta + days)   # number of tests
# size of our test 
test_size = number_of_items - 733

# five day blocks in a list
Xtrain = []    
# the values of the day we are trying to predict
Ytrain = []                                                   



# Splits the array of closing prices up into multiple arrays, each holding five days worth of prices
# then transposes each array and finally appends them onto one large matrix
for i in range(test_size): 
    # finding the predict day
    predict_day = rawdata.head(days + i + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    last_day = rawdata.head(days + i).tail(1).as_matrix().transpose().tolist()[0][0]
    # Converting and adding data to arrays
    res = AvgPrevious(rawdata.head(days + i).tail(days).as_matrix().transpose().tolist()[0])
    Xtrain.append(res)
    Ytrain.append((predict_day - last_day))

# converts python arrays into numpy arrays
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)

print(Xtrain)

154.86999999999998
-10.329999999999984
-15.869999999999976
154.86999999999998
132.85999999999996
-15.869999999999976
-27.649999999999963
132.85999999999996
133.46
5.539999999999992
-16.470000000000013
133.46
121.68
-16.470000000000013
-23.930000000000007
121.68
94.97999999999999
22.010000000000005
10.230000000000004
94.97999999999999
87.52
10.230000000000004
24.980000000000004
87.52
93.42999999999999
11.780000000000001
4.320000000000007
93.42999999999999
108.17999999999999
4.320000000000007
7.730000000000004
108.17999999999999
90.29
7.459999999999994
22.209999999999994
90.29
93.7
22.209999999999994
18.599999999999994
93.7
127.25
-14.75
-11.340000000000003
127.25
123.64
-11.340000000000003
-12.14
123.64
119.32
-3.4099999999999966
-7.019999999999996
119.32
118.52
-7.019999999999996
-4.950000000000003
118.52
108.69
3.6099999999999994
2.8100000000000023
108.69
110.75999999999999
2.8100000000000023
1.9100000000000108
110.75999999999999
110.7
0.7999999999999972
2.8699999999999903
110.7
109.8

1126.89
2.5399999999999636
-171.04000000000008
1126.89
1214.37
-171.04000000000008
-136.08999999999992
1214.37
782.27
173.58000000000004
261.05999999999995
782.27
817.22
261.05999999999995
333.95000000000005
817.22
1130.81
-87.4799999999999
-52.52999999999997
1130.81
1203.7
-52.52999999999997
-158.59000000000015
1203.7
1113.23
-34.950000000000045
37.940000000000055
1113.23
1007.1699999999998
37.940000000000055
-177.7199999999998
1007.1699999999998
1224.0600000000002
-72.8900000000001
-178.95000000000027
1224.0600000000002
1008.4000000000003
-178.95000000000027
-310.1700000000003
1008.4000000000003
939.0499999999997
106.06000000000017
-109.59999999999968
939.0499999999997
807.8299999999997
-109.59999999999968
-11.959999999999695
807.8299999999997
613.7900000000002
215.65999999999985
84.43999999999983
613.7900000000002
711.4300000000002
84.43999999999983
181.75999999999988
711.4300000000002
567.01
131.22000000000003
228.86
567.01
664.33
228.86
324.17999999999995
664.33
893.51
-97.6399999

486.45
15.110000000000014
11.720000000000027
486.45
463.32
16.319999999999993
38.24000000000001
463.32
459.93
38.24000000000001
35.839999999999975
459.93
523.48
-21.920000000000016
-25.310000000000002
523.48
521.0799999999999
-25.310000000000002
-33.15999999999991
521.0799999999999
494.78000000000003
3.3899999999999864
0.9899999999999523
494.78000000000003
486.93000000000006
0.9899999999999523
4.369999999999948
486.93000000000006
493.36999999999995
2.400000000000034
-5.449999999999932
493.36999999999995
496.74999999999994
-5.449999999999932
3.7100000000000364
496.74999999999994
480.07000000000005
7.849999999999966
11.229999999999961
480.07000000000005
489.23
11.229999999999961
-27.78000000000003
489.23
494.68
-3.3799999999999955
5.779999999999973
494.68
455.67
5.779999999999973
2.930000000000007
455.67
509.61999999999995
-9.159999999999968
-48.16999999999996
509.61999999999995
506.77
-48.16999999999996
-70.38
506.77
422.44
39.00999999999999
36.160000000000025
422.44
400.22999999999996


503.2200000000001
477.0199999999999
6.32000000000005
7.810000000000059
477.0199999999999
474.4699999999999
7.810000000000059
0.13000000000010914
474.4699999999999
486.32
-1.490000000000009
-4.0400000000000205
486.32
478.64000000000004
-4.0400000000000205
-3.3800000000000523
478.64000000000004
479.72999999999996
2.5500000000000114
-5.129999999999939
479.72999999999996
480.38999999999993
-5.129999999999939
-1.0299999999999159
480.38999999999993
466.9200000000001
7.67999999999995
8.339999999999918
466.9200000000001
471.0200000000001
8.339999999999918
8.729999999999905
471.0200000000001
475.91999999999996
-0.6599999999999682
3.4400000000000546
475.91999999999996
476.30999999999995
3.4400000000000546
1.4400000000000546
476.30999999999995
483.46000000000004
-4.100000000000023
-3.7100000000000364
483.46000000000004
481.46000000000004
-3.7100000000000364
-2.4600000000000364
481.46000000000004
480.14
-0.38999999999998636
-2.3899999999999864
480.14
481.39
-2.3899999999999864
-3.5
481.39
475.75
2

-6.649999999999977
301.0
291.98999999999995
-6.649999999999977
-10.109999999999957
291.98999999999995
292.32000000000005
2.0299999999999727
-6.980000000000075
292.32000000000005
288.86000000000007
-6.980000000000075
-2.470000000000084
288.86000000000007
276.3299999999999
9.010000000000048
5.550000000000068
276.3299999999999
280.8399999999999
5.550000000000068
9.750000000000057
280.8399999999999
278.42
3.4599999999999795
7.96999999999997
278.42
282.62
7.96999999999997
2.8799999999999955
282.62
290.9
-4.509999999999991
-0.3100000000000023
290.9
285.81
-0.3100000000000023
-29.50999999999999
285.81
294.78999999999996
-4.199999999999989
-9.289999999999964
294.78999999999996
265.59
-9.289999999999964
-4.659999999999968
265.59
280.41
5.089999999999975
-24.110000000000014
280.41
285.04
-24.110000000000014
-23.29000000000002
285.04
227.10000000000002
29.19999999999999
33.829999999999984
227.10000000000002
227.92000000000002
33.829999999999984
32.099999999999966
227.92000000000002
265.56
-4.6299

0.2500000000000284
229.55999999999997
229.96000000000004
0.339999999999975
-0.870000000000033
229.96000000000004
230.68000000000004
-0.870000000000033
2.2899999999999636
230.68000000000004
227.88
1.210000000000008
1.9300000000000068
227.88
231.04
1.9300000000000068
0.45000000000001705
231.04
230.53
-0.7199999999999989
2.4399999999999977
230.53
229.05
2.4399999999999977
2.1599999999999966
229.05
236.13
-3.1599999999999966
-4.639999999999986
236.13
235.85
-4.639999999999986
-8.759999999999991
235.85
230.01000000000002
1.4799999999999898
1.1999999999999886
230.01000000000002
225.89000000000001
1.1999999999999886
4.72999999999999
225.89000000000001
230.93
0.28000000000000114
-3.8400000000000034
230.93
234.46
-3.8400000000000034
-4.180000000000007
234.46
222.97
4.1200000000000045
7.650000000000006
222.97
222.63
7.650000000000006
11.900000000000006
222.63
234.15
-3.530000000000001
-3.8700000000000045
234.15
238.4
-3.8700000000000045
-3.2600000000000193
238.4
229.94
0.3400000000000034
4.59000

In [5]:
# number of evalutaion data points
eval_size = number_of_items - test_size

# evaluation input data
XEvaluation = []
# evaluation output data
YEvaluation = []

# appending evaluation data to appropriate arrays
for i in range(eval_size):
    # finding the day we want to predict a week out
    predict_day = rawdata.head(days + i + test_size + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    
    # finding the last day out of the 5 we are using as input to later calculate the percent difference 
    # of the change.  The percentage change is not the absolute value as we want to calculate negative percentages as well
    last_day = rawdata.head(days + i + test_size).tail(1).as_matrix().transpose().tolist()[0][0]
    
    #result after running the AvgPrevious function on the 5 days used as input
    res =rawdata.head(days + i + test_size).tail(days).as_matrix().transpose().tolist()[0]
    
    XEvaluation.append(res)
    YEvaluation.append((predict_day - last_day))
    
XEvaluation = np.array(XEvaluation)
YEvaluation = np.array(YEvaluation)

print(XEvaluation)
print(YEvaluation)

[[  387.49   402.97   391.73   392.15   394.97]
 [  402.97   391.73   392.15   394.97   380.29]
 [  391.73   392.15   394.97   380.29   379.47]
 ...
 [10931.4  10868.4  11359.4  11259.4  11171.4 ]
 [10868.4  11359.4  11259.4  11171.4  11440.7 ]
 [11359.4  11259.4  11171.4  11440.7  11786.3 ]]
[-6.76307609e-02  2.38712493e-02  1.83158712e-02 -4.59470944e-03
  2.08432903e-02  1.04431651e-03  4.20179241e-03  3.06563605e-02
 -2.61820097e-02 -5.95950658e-03  3.91466340e-02  7.51663679e-02
  6.68182613e-02  7.72043486e-02  8.32772867e-02  1.01143547e-01
  8.68128995e-02  1.03623387e-01  7.19462170e-02  8.58023986e-02
  3.14921329e-02  2.03082716e-02  5.11141471e-03  2.62871688e-02
 -1.07278276e-02 -1.22260669e-02 -1.14233493e-04  3.30483545e-02
 -2.26420434e-03 -6.85402585e-03 -5.16133742e-02 -7.97613401e-02
 -6.32557455e-02 -5.64298127e-02 -5.10906587e-02 -2.20074242e-02
 -1.08359504e-02  2.54926605e-02  2.68451484e-02  1.53359417e-02
  5.09076938e-03  6.86131037e-03  5.15575166e-03  8.2972

In [27]:
model = Sequential()

model.add(Dense(1,input_shape=(5,)))
model.add(Activation('relu'))
model.add(Dense(1, activation='tanh'))
model.add(Dropout(5.0))

model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

H = model.fit(Xtrain, Ytrain, epochs=12)

Epoch 1/12
1000/1000 [==============================] - 0s 296us/step - loss: 30.3104 - acc: 0.0020
Epoch 2/12
1000/1000 [==============================] - 0s 109us/step - loss: 30.3104 - acc: 0.0020
Epoch 3/12
1000/1000 [==============================] - 0s 105us/step - loss: 30.3104 - acc: 0.0020
Epoch 4/12
1000/1000 [==============================] - 0s 100us/step - loss: 30.3104 - acc: 0.0020
Epoch 5/12
1000/1000 [==============================] - 0s 113us/step - loss: 30.3104 - acc: 0.0020
Epoch 6/12
1000/1000 [==============================] - 0s 196us/step - loss: 30.3104 - acc: 0.0020
Epoch 7/12
1000/1000 [==============================] - 0s 136us/step - loss: 30.3104 - acc: 0.0020
Epoch 8/12
1000/1000 [==============================] - 0s 139us/step - loss: 30.3104 - acc: 0.0020
Epoch 9/12
1000/1000 [==============================] - 0s 135us/step - loss: 30.3104 - acc: 0.0020 ETA: 0s - loss: 27.5286 - acc: 0.0024 
Epoch 10/12
1000/1000 [==============================] - 0s 1

In [11]:
Test = [8070.8, 8891.21, 8516.24, 9477.84, 10016.49]
XVal = np.array([Test])
r = model.predict(XVal)
result = r[0][0]

if result >= 1 - 2/3:
    print("it will go up by 3%")
elif result > 0 - 2/3 and result < 1 - 2/3:
    print("it will stay between 3%")
else:
    print("it will go down by 3%")

print(result)

it will stay between 3%
0.00040043562


In [16]:
score = model.evaluate(XEvaluation,YEvaluation,batch_size=100, verbose=1)
test_loss = score[0]
print("The evaluation loss is: " + str(test_loss))

733/733 [==============================] - 0s 21us/step
The evaluation loss is: 0.0118102734421836


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1)                 6         
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1)                 0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [29]:
rawdata = pd.read_csv('data/bitcoin.csv',usecols=[3,8])
data = rawdata.as_matrix()
print(np.argwhere(data=='2016-10-08')[0][0])

res = getEvalData(data,'2015-10-15',predict_delta)

evaluation = np.array([res[0]])
print(evaluation)
print(AvgPrevious(evaluation[0]))
#model.predict(evaluation)
#print(rawdata[0].loc['2013-04-28'])

1259
895
[[244.94 247.05 245.31 249.51 251.99]]
[  2.11 244.94   0.37 249.14   2.85]
